In [11]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
import time
import statistics

import xgboost as xgb
import dask.array as da
import dask.distributed

In [12]:
times=[]

In [13]:
client = Client('172.31.45.128:8786')
client.restart()
print(client)

<Client: 'tcp://172.31.45.128:8786' processes=0 threads=0, memory=0 B>


In [14]:
data = dd.read_csv("/home/ubuntu/data.csv", sample=2**20)
data = data.persist()

In [15]:
train, test = data.random_split([0.7, 0.3], random_state=42)

In [16]:
X_train = train.drop(labels=["tip_amount"], axis=1).copy()
Y_train = train["tip_amount"].copy()

In [17]:
X_test = test.drop(labels=["tip_amount"], axis=1).copy()
Y_test = test["tip_amount"].copy()

# 10 Workers

In [18]:
t0 = time.time()
dtrain = xgb.dask.DaskDMatrix(client,X_train,Y_train)
output = xgb.dask.train(client
               ,{"verbosity": 2
                 , "tree_method": "hist"
                 , "objective": "reg:squarederror"}, dtrain
               , num_boost_round=6, evals=[(dtrain, "train")],)
t1 = time.time()
times.append(t1-t0)

In [19]:
t0 = time.time()
prediction = xgb.dask.predict(client, output['booster'], X_test)
t1 = time.time()
times.append(t1-t0)

In [20]:
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((Y_test_array - prediction) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")
print(f"R^2 Score: {r2_score_result}")

Mean Squared Error: 0.017532391298928442
Root Mean Squared Error: 0.13240993655662117
Mean Absolute Error: 0.046429214322379835
R^2 Score: -1.2528115416301606


In [21]:
times

[10.303650140762329, 0.0752871036529541]

# 6 Workers

In [22]:
t0 = time.time()
dtrain = xgb.dask.DaskDMatrix(client,X_train,Y_train)
output = xgb.dask.train(client
               ,{"verbosity": 2
                 , "tree_method": "hist"
                 , "objective": "reg:squarederror"}, dtrain
               , num_boost_round=6, evals=[(dtrain, "train")],)
t1 = time.time()
times.append(t1-t0)

In [23]:
t0 = time.time()
prediction = xgb.dask.predict(client, output['booster'], X_test)
t1 = time.time()
times.append(t1-t0)

In [24]:
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((Y_test_array - prediction) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")
print(f"R^2 Score: {r2_score_result}")

Mean Squared Error: 0.017536946858902676
Root Mean Squared Error: 0.1324271379246062
Mean Absolute Error: 0.04644523558185246
R^2 Score: -1.2533969049108196


In [25]:
times

[10.303650140762329,
 0.0752871036529541,
 8.947437524795532,
 0.10448265075683594]

# 2 Workers

In [26]:
t0 = time.time()
dtrain = xgb.dask.DaskDMatrix(client,X_train,Y_train)
output = xgb.dask.train(client
               ,{"verbosity": 2
                 , "tree_method": "hist"
                 , "objective": "reg:squarederror"}, dtrain
               , num_boost_round=6, evals=[(dtrain, "train")],)
t1 = time.time()
times.append(t1-t0)

In [27]:
t0 = time.time()
prediction = xgb.dask.predict(client, output['booster'], X_test)
t1 = time.time()
times.append(t1-t0)

In [28]:
Y_test_array = Y_test.to_dask_array(lengths=True)

# Calculate Mean Squared Error
mse = da.mean((prediction - Y_test_array) ** 2)
rmse = da.sqrt(mse)

# Calculate Mean Absolute Error
mae = da.mean(da.absolute(prediction - Y_test_array))

# Calculate R^2 Score
ss_res = da.sum((Y_test_array - prediction) ** 2)
ss_tot = da.sum((Y_test_array - da.mean(Y_test_array)) ** 2)
r2_score = 1 - (ss_res / ss_tot)

# Compute the results
mse_result = mse.compute()
rmse_result = rmse.compute()
mae_result = mae.compute()
r2_score_result = r2_score.compute()

print(f"Mean Squared Error: {mse_result}")
print(f"Root Mean Squared Error: {rmse_result}")
print(f"Mean Absolute Error: {mae_result}")
print(f"R^2 Score: {r2_score_result}")

Mean Squared Error: 0.017532796221959506
Root Mean Squared Error: 0.1324114655985633
Mean Absolute Error: 0.046429648360056135
R^2 Score: -1.2528635719129841


In [29]:
times

[10.303650140762329,
 0.0752871036529541,
 8.947437524795532,
 0.10448265075683594,
 20.11827850341797,
 0.06145668029785156]